In [1]:
import pandas as pd
df = pd.read_csv("anr.csv")

In [2]:
df.head()

,symbol,mu,sigma,entropy,position
0,MS US Equity,0.271594,0.017980,0.407194,1
1,MA US Equity,0.076093,0.005824,0.686622,1
2,ELAN US Equity,0.198331,0.006827,0.415581,1
3,C US Equity,0.227062,0.023382,0.508636,1
4,CTL US Equity,0.360252,0.100368,0.013464,1


In [52]:
from ortools.linear_solver import pywraplp
import requests
import json

# Instantiate a Glop solver, naming it SolveStigler.
solver = pywraplp.Solver('SolveStigler', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

nassets = df.shape[0]
w = [[]] * nassets

# Objective: minimize the sum of (price-normalized) foods.
objective = solver.Objective()
for i in range(0, nassets):
  w[i] = solver.NumVar(0.0, 0.2, df['symbol'][i])
  objective.SetCoefficient(w[i], df['mu'][i])
objective.SetMaximization()

constraint0 = solver.Constraint(1, 1)
constraint1 = solver.Constraint(-solver.infinity(), 0.1)
constraint2 = solver.Constraint(0.85, solver.infinity())
for i in range(0, nassets):
    constraint0.SetCoefficient(w[i], 1)
    constraint1.SetCoefficient(w[i], df['sigma'][i])
    constraint2.SetCoefficient(w[i], df['entropy'][i])

status = solver.Solve()

capital = 1000
apiKey = "OFWNX9PJR40H" # + (arba3a) (sifer) (awal 7arf min 3elit tom)

if status == solver.OPTIMAL:
    er = 0.0
    for i in range(0, nassets):
        if w[i].solution_value() > 0.0:
            symbol = str(w[i]).split(' ')[0]
            url = "https://api.tdameritrade.com/v1/marketdata/chains?apikey="+apiKey+"&symbol="+symbol+"&contractType=CALL&strikeCount=1&range=OTM&fromDate=2019-09-01"
            resp = requests.get(url)
            #print(resp.content)
            obj = json.loads(resp.content)
            last_p = obj['underlyingPrice']
            exp_dates = obj['callExpDateMap'].keys()
            print(exp_dates)
            strikes = obj['callExpDateMap'][list(exp_dates)[0]].keys()
            print(strikes)
            option_p = obj['callExpDateMap'][list(exp_dates)[0]][list(strikes)[0]][0]['last']
            er += w[i].solution_value()*df['mu'][i]
            print("%s [%s] @ %.2f = %0.4f with E[%.2f] # %.2f" % (w[i], df['position'][i], last_p, w[i].solution_value(), df['mu'][i], (capital*w[i].solution_value())/option_p))
    print("expected returns =", er)

dict_keys(['2019-09-20:217', '2020-01-17:336', '2021-01-15:700'])
dict_keys(['18.0'])
AMRN US Equity [1] @ 17.65 = 0.2000 with E[1.19] # 54.79
dict_keys(['2019-09-20:217', '2020-01-17:336', '2020-06-19:490', '2021-01-15:700'])
dict_keys(['1120.0'])
GOOGL US Equity [1] @ 1120.70 = 0.0392 with E[0.16] # 0.40
dict_keys(['2020-01-17:336', '2021-01-15:700'])
dict_keys(['15.0'])
IMMU US Equity [1] @ 14.05 = 0.0506 with E[0.68] # 10.76
dict_keys(['2019-09-20:217', '2019-10-18:245', '2020-01-17:336', '2020-06-19:490', '2021-01-15:700'])
dict_keys(['170.0'])
BABA US Equity [1] @ 166.17 = 0.2000 with E[0.21] # 13.89
dict_keys(['2020-01-17:336', '2020-06-19:490', '2021-01-15:700'])
dict_keys(['1610.0'])
AMZN US Equity [1] @ 1609.19 = 0.2000 with E[0.28] # 0.99
dict_keys(['2020-01-17:336', '2021-01-15:700'])
dict_keys(['70.0'])
CVS US Equity [1] @ 69.90 = 0.1102 with E[0.34] # 16.02
dict_keys(['2020-01-17:336', '2021-01-15:700'])
dict_keys(['17.0'])
ET US Equity [1] @ 15.03 = 0.2000 with E[0.45] #